# **JuliaCFD**

In [5]:
using BenchmarkTools
using Pkg; Pkg.add("PyCall"); 
# Pkg.build("PyPlot")
using PyPlot
using PyCall
jtplot = pyimport("jupyterthemes")
jtplot.style(grid=false)
rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
rcParams["font.size"] = 16
rcParams["font.family"] = "serif"
rcParams["text.usetex"] = true

   Updating registry at `C:\Users\godot\.julia\registries\General`
   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
No Changes to `C:\Users\godot\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\godot\.julia\environments\v1.5\Manifest.toml`


LoadError: KeyError: key "style" not found

### von Neumann Stability Analysis

Consider the heat equation in one dimension:

$$ \frac{\partial u}{\partial t} + \nu \frac{\partial^2 u}{\partial x^2} = 0 $$

Now separate the variables via Fourier series: $ u(t_n, x) = \sum_{j=0}^N \hat u(t) \exp(ik_j x) $.

## Vorticity-Stream Function Formulation

#### Governing Equations:

Taking the curl of the incompressible Navier-Stokes equations:

$$ \nabla \times \rho\frac{\partial \mathbf u}{\partial t} +  \nabla \times \rho\left(\mathbf u \cdot \nabla\right)\mathbf u  = -\nabla \times \nabla p + \nabla \times \mu\nabla^2\mathbf u $$

$$ \implies \frac{\partial \omega}{\partial t} + \left(\mathbf u \cdot \nabla \right)\omega = \nu\nabla^2 \omega$$

Discretisation (FTCS):

$$ \frac{\omega^{n+1}_{i,j} - \omega^n_{i,j}}{\Delta t} + u^n_{i,j}\frac{\omega^n_{i+1, j} - \omega^n_{i-1, j}}{2\Delta x} + v_{i,j}\frac{\omega^n_{i, j+1} - \omega^n_{i, j-1}}{2\Delta y} = \nu\left(\frac{\omega^n_{i+1, j} - 2\omega^n_{i, j} + \omega^n_{i-1, j}}{\Delta x^2} + \frac{\omega^n_{i, j+1} - 2\omega^n_{i, j} + \omega^n_{i, j-1}}{\Delta y^2}\right) $$

The Cauchy-Riemann conditions for the streamfunction-velocity relation are: 

$$ u = -\frac{\partial \psi}{\partial y}, ~~~ v = \frac{\partial \psi}{\partial x} $$

Inserting this into the definition of the vorticity in 2D:

$$ \omega = \frac{\partial u}{\partial y} - \frac{\partial v}{\partial x} $$

Which gives the following Poisson equation:

$$ \nabla^2\psi = -\omega $$

As the setup is explicit, the CFL condition must be satisfied with the convection condition:

$$ \frac{\nu\Delta t}{\Delta x^2} + \frac{\nu\Delta t}{\Delta y^2} \leq 1, ~~~ \frac{U\Delta t}{\Delta x} = 1 $$

Using the discrete version of the Cauchy-Riemann equations, taking $\Delta x = \Delta y$ and substituting:

$$ \omega^{n+1}_{i,j} = \omega^n_{i,j} - \frac{1}{4\Delta x\Delta y}\left[\left(\psi^n_{i+1,j} - \psi^n_{i-1,j}\right)\left(\omega^n_{i+1,j} - \omega^n_{i-1,j}\right) + \left(\psi^n_{i,j+1} - \psi^n_{i,j-1}\right)\left(\omega^n_{i,j+1} - \omega^n_{i,j-1}\right)\right] + \frac{\nu\Delta t}{\Delta x^2}\left(\omega^n_{i+1, j} + \omega^n_{i-1, j} - 4\omega^n_{i,j} + \omega^n_{i, j+1} + \omega^n_{i, j-1}\right)  $$

$$ \frac{\psi^{n+1}_{i+1,j}-2\psi^{n+1}_{i,j}+\psi^{n+1}_{i-1,j}}{\Delta x^2_1}+\frac{\psi^{n+1}_{i,j+1}-2\psi^{n+1}_{i,j}+\psi^{n+1}_{i,j-1}}{\Delta x^2_2}=-\omega^{n+1}_{i,j} $$

#### Boundary Conditions:

No-slip boundary on the walls:

$$ \vec U \cdot \vec t = 0 \implies \frac{\partial \psi}{\partial x} = \frac{\partial \psi}{\partial y} = \text{constant} $$



# VOF Solver

Governing equations:
$$ \nabla \cdot \mathbf u = 0 $$
$$ \rho\frac{\partial \mathbf u}{\partial t} + \rho\left(\nabla \cdot \mathbf u \right)\mathbf u  = -\nabla p + \rho \mathbf g + \mu\nabla^2\mathbf u $$

Discretisation:

$$ \frac{\mathbf{u}^* - \mathbf{u}^n}{\Delta t} = -\mathbf A^n - $$ 